In [3]:
"""
Exercise 02: Permission-Based Tools - Starter Code
===================================================
Build an agent with tier-based tool access.

LEARNING GOALS:
- Import and use @wrap_model_call decorator
- Filter tools based on state values
- Use request.override() to modify requests
"""

import os
from typing import Callable
from dotenv import load_dotenv

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool

load_dotenv()


# =============================================================================
# TODO 1: Import Model Call Wrapper Components
# =============================================================================
# Import from langchain.agents.middleware:
# - wrap_model_call (decorator)
# - ModelRequest (input type)
# - ModelResponse (output type)
# =============================================================================

from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse

In [4]:

# =============================================================================
# TODO 2: Create Tools by Tier
# =============================================================================
# Create tools for each subscription tier:
# - Free: basic operations accessible to all
# - Pro: advanced features for paid users
# - Enterprise: full capabilities
#
# EXPERIMENT: What tool descriptions help the agent recommend upgrades?
# =============================================================================

@tool
def run_basic_query(query: str) -> str:
    """Run a basic data query. Available to all users."""
    return f"Basic query executed: '{query}'. Results: Found 42 matching records."


@tool
def run_advanced_analysis(dataset: str, analysis_type: str) -> str:
    """Run advanced analysis. PRO tier and above only."""
    return f"Advanced {analysis_type} analysis completed on '{dataset}'. Generated statistical insights and trends."


@tool
def create_visualization(data: str, chart_type: str) -> str:
    """Create a visualization. PRO tier and above only."""
    return f"Created {chart_type} visualization from data: '{data}'. Chart saved successfully."


@tool
def export_data(format: str) -> str:
    """Export data to file. ENTERPRISE tier only."""
    return f"Data exported successfully to {format} format. File ready for download."


@tool
def sync_external(destination: str) -> str:
    """Sync to external system. ENTERPRISE tier only."""
    return f"Successfully synced data to external destination: {destination}. Sync completed."


In [5]:

# =============================================================================
# TODO 3: Define Tool Groups
# =============================================================================
# Create lists of tools for each tier:
# - FREE_TOOLS: basic only
# - PRO_TOOLS: basic + advanced features
# - ENTERPRISE_TOOLS: all tools
# =============================================================================

FREE_TOOLS = [run_basic_query]

PRO_TOOLS = [run_basic_query, run_advanced_analysis, create_visualization]

ENTERPRISE_TOOLS = [run_basic_query, run_advanced_analysis, create_visualization, export_data, sync_external]


In [6]:
# =============================================================================
# TODO 4: Implement Permission Middleware
# =============================================================================
# Create a function decorated with @wrap_model_call that:
# 1. Receives request (ModelRequest) and handler (Callable)
# 2. Reads user_tier from request.state.get("user_tier", "free")
# 3. Selects the appropriate tool list based on tier
# 4. Uses request.override(tools=...) to modify available tools
# 5. Returns handler(modified_request)
#
# EXPERIMENT: What if you add a message explaining available features?
# EXPERIMENT: What happens if user asks for unavailable feature?
# =============================================================================

@wrap_model_call
def permission_middleware(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    user_tier = request.state.get("user_tier", "free").lower()
    
    if user_tier == "enterprise":
        tools = ENTERPRISE_TOOLS
    elif user_tier == "pro":
        tools = PRO_TOOLS
    else:
        tools = FREE_TOOLS
    
    modified_request = request.override(tools=tools)
    return handler(modified_request)

In [7]:
# =============================================================================
# TODO 5: Create the Agent
# =============================================================================
# Create an agent with:
# - All tools (middleware will filter them)
# - Your permission middleware
# - A helpful system prompt that handles unavailable features gracefully
# =============================================================================

model = init_chat_model("gpt-4o-mini", model_provider="openai")

agent = create_agent(
    model=model,
    tools=FREE_TOOLS,
    middleware=[permission_middleware],
    system_prompt="You are a helpful data analysis assistant. Your available tools depend on the user's subscription tier. If a user requests a feature that's not available in their tier, politely explain that it requires an upgrade and suggest the appropriate tier. Always work with the tools available to the user.",
    name="permission_based_agent"
)

In [8]:
# =============================================================================
# Testing the Agent with Different Tiers
# =============================================================================

print("Testing Permission-Based Agent\n")
print("=" * 60)

# Test 1: Free tier user - basic query only
print("\n1. Testing FREE tier user:")
print("-" * 60)
result_free = agent.invoke({
    "messages": [{"role": "user", "content": "Run a query for sales data"}],
    "state": {"user_tier": "free"}
})
print(f"User: Run a query for sales data")
print(f"Agent: {result_free['messages'][-1].content}\n")

# Test 2: Free tier user requesting unavailable feature
print("\n2. Testing FREE tier user requesting unavailable feature:")
print("-" * 60)
result_free_upgrade = agent.invoke({
    "messages": [{"role": "user", "content": "Create a visualization of my data"}],
    "state": {"user_tier": "free"}
})
print(f"User: Create a visualization of my data")
print(f"Agent: {result_free_upgrade['messages'][-1].content}\n")

# Test 3: Pro tier user - advanced features available
print("\n3. Testing PRO tier user:")
print("-" * 60)
result_pro = agent.invoke({
    "messages": [{"role": "user", "content": "Create a bar chart visualization"}],
    "state": {"user_tier": "pro"}
})
print(f"User: Create a bar chart visualization")
print(f"Agent: {result_pro['messages'][-1].content}\n")

# Test 4: Enterprise tier user - all features available
print("\n4. Testing ENTERPRISE tier user:")
print("-" * 60)
result_enterprise = agent.invoke({
    "messages": [{"role": "user", "content": "Export my data to CSV and sync to S3"}],
    "state": {"user_tier": "enterprise"}
})
print(f"User: Export my data to CSV and sync to S3")
print(f"Agent: {result_enterprise['messages'][-1].content}\n")

Testing Permission-Based Agent


1. Testing FREE tier user:
------------------------------------------------------------
User: Run a query for sales data
Agent: Could you please specify the details of the sales data you would like to query? For example, do you want data from a specific time period, for certain products, or any other criteria? Let me know so I can help you effectively!


2. Testing FREE tier user requesting unavailable feature:
------------------------------------------------------------
User: Create a visualization of my data
Agent: Creating visualizations requires an upgrade to a higher subscription tier that includes visualization tools. I recommend upgrading to access those features for a more comprehensive analysis experience. 

For now, I can assist you by running basic data queries. If there's any specific data-related question you have or if you'd like me to run a basic query, please let me know!


3. Testing PRO tier user:
--------------------------------------